In [1]:
!pip install transformers datasets torch --quiet
!pip install --upgrade datasets huggingface_hub --quiet
!pip install fsspec==2023.6.0 --quiet

In [2]:
from datasets import load_dataset
from huggingface_hub import login
from google.colab import userdata

# SETAR A CHAVE DO HUGGINGFACE
login(token = userdata.get('HF_TOKEN'))

# dataset IMDb
dataset = load_dataset("imdb")

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [3]:
from transformers import pipeline

sentiment_analyzer = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cuda:0


In [4]:
# Pega 5 exemplos de teste
test_texts = dataset["test"]["text"][:5]

# Faz a análise
results = sentiment_analyzer(test_texts)

for text, result in zip(test_texts, results):
    print(f"Texto: {text[:250]}...")
    print(f"Sentimento: {result['label']}, score: {result['score']:.4f}")
    print()


Texto: I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prost...
Sentimento: NEGATIVE, score: 0.9996

Texto: Worth the entertainment value of a rental, especially if you like action movies. This one features the usual car chases, fights with the great Van Damme kick style, shooting battles with the 40 shell load shotgun, and even terrorist style bombs. All ...
Sentimento: NEGATIVE, score: 0.6171

Texto: its a totally average film with a few semi-alright action sequences that make the plot seem a little better and remind the viewer of the classic van dam films. parts of the plot don't make sense and seem to be added in to use up time. the end plot is...
Sentimento: NEGATIVE, score: 0.9997

Texto: STAR RATING: ***** Saturday Night **** Friday Night *** Friday Morning ** Sunday Night * Monday Mor

In [5]:
# previsão
test_texts = dataset["test"]["text"]
test_labels = dataset["test"]["label"]

max_length = 512

test_texts_short = [text[:max_length] for text in test_texts]
results = sentiment_analyzer(test_texts_short, batch_size=32)

# Traduz rótulos de comparação
def label_to_int(label):
    return 1 if label == "POSITIVE" else 0

pred_labels = [label_to_int(r["label"]) for r in results]

# acurácia
correct = sum([p == t for p, t in zip(pred_labels, test_labels)])
accuracy = correct / len(test_labels)
print(f"Acurácia no conjunto de teste: {accuracy:.4f}")


Acurácia no conjunto de teste: 0.8283
